In [1]:
import os

import bids

from bids import BIDSLayout
from bids import BIDSValidator



In [2]:
path = 'F:\BIDS_face-expr-nf'
layout = BIDSLayout(path)


<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\direi\AppData\Local\Temp\ipykernel_540\1215669490.py:1: SyntaxWarning: invalid escape sequence '\B'
  path = 'F:\BIDS_face-expr-nf'


In [3]:
# get events tsv files from bids dataset in path
events = layout.get(suffix='events', extension='tsv', return_type='file', subject='01')

print(events)

['F:\\BIDS_face-expr-nf\\sub-01\\ses-01\\func\\sub-01_ses-01_task-auditoryfeedback_run-01_events.tsv', 'F:\\BIDS_face-expr-nf\\sub-01\\ses-01\\func\\sub-01_ses-01_task-faceexplocalizer_run-01_events.tsv', 'F:\\BIDS_face-expr-nf\\sub-01\\ses-01\\func\\sub-01_ses-01_task-nofeedback_run-01_events.tsv', 'F:\\BIDS_face-expr-nf\\sub-01\\ses-01\\func\\sub-01_ses-01_task-visualfeedback_run-01_events.tsv']


In [4]:
# read .prt file

import bvbabel

FILE = "C:/Users/direi/Documents/GitHub/OpenNeuro-utils/t.prt"

# =============================================================================
# Load VOI
header, data = bvbabel.prt.read_prt(FILE)

# Print header information
print("\nPRT header")
for key, value in header.items():
    print("  ", key, ":", value)

# Print data
print("\nPRT data")
for d in data:
    for key, value in d.items():
        print("  ", key, ":", value)
    print("")




PRT header
   FileVersion : 2
   ResolutionOfTime : Volumes
   Experiment : Untitled
   BackgroundColor : 0 0 0
   TextColor : 255 255 255
   TimeCourseColor : 255 255 30
   TimeCourseThick : 2
   ReferenceFuncColor : 30 200 30
   ReferenceFuncThick : 2
   NrOfConditions : 3

PRT data
   NameOfCondition : fixation
   NrOfOccurances : 9
   Time start : [  1  33  65  97 129 161 193 225 257]
   Time stop : [  8  40  72 104 136 168 200 232 264]
   Color : [195 195 195]

   NameOfCondition : faces
   NrOfOccurances : 4
   Time start : [  9  73 137 201]
   Time stop : [ 32  96 160 224]
   Color : [255   0   0]

   NameOfCondition : objects
   NrOfOccurances : 4
   Time start : [ 41 105 169 233]
   Time stop : [ 64 128 192 256]
   Color : [  0   0 255]

Finished.


In [24]:
# Place all data in a dataframe where each row is a trial
import pandas as pd
df = pd.DataFrame(columns = ['trial_type','onset', 'duration'])

for d in data:
    for key, value in d.items():
        # iterate over 'Time start' key
        if key == 'Time start':
            # add value to trial_data
            df =pd.concat([df, pd.DataFrame({'trial_type' : d['NameOfCondition'], 'duration' : 0, 'onset' : value})])




In [27]:
df.sort_values("onset")

,onset,duration,trial_type
0,1,0,fixation
0,9,0,faces
1,33,0,fixation
0,41,0,objects
2,65,0,fixation
1,73,0,faces
3,97,0,fixation
1,105,0,objects
4,129,0,fixation
2,137,0,faces


In [ ]:
# Create a list of dictionaries with the onset, duration and trial_type
events = []
for i, row in df.iterrows():
    events.append({
        "onset": row["onset"],
        "duration": row["duration"],
        "trial_type": row["name"]
    })